In [1]:
import pandas as pd

In [2]:
df=pd.read_csv(".\Agitation.csv")

In [3]:
df.head()

,timestamp (ET),agitimestamp (ET),Location,level,Observation,Behaviour
0,12/10/2016 21:02,12/10/2016 21:00,Kitchen,5,Withdrawn,Normal
1,12/15/2016 3:22,12/15/2016 3:21,Kitchen,3,"Vocal1, Withdrawn",Normal
2,12/15/2016 19:14,12/16/2016 2:20,Other,3,Withdrawn,Normal
3,12/19/2016 1:57,12/19/2016 1:55,Other,4,Withdrawn,Normal
4,12/28/2016 20:36,12/28/2016 19:00,Other,4,"Vocal2, Withdrawn",Normal


In [4]:
df['Behaviour'] = df['Behaviour'].map({"Normal" : 1,
                                     "Agitation" : 0})

In [5]:
df.head()

,timestamp (ET),agitimestamp (ET),Location,level,Observation,Behaviour
0,12/10/2016 21:02,12/10/2016 21:00,Kitchen,5,Withdrawn,1
1,12/15/2016 3:22,12/15/2016 3:21,Kitchen,3,"Vocal1, Withdrawn",1
2,12/15/2016 19:14,12/16/2016 2:20,Other,3,Withdrawn,1
3,12/19/2016 1:57,12/19/2016 1:55,Other,4,Withdrawn,1
4,12/28/2016 20:36,12/28/2016 19:00,Other,4,"Vocal2, Withdrawn",1


In [6]:
X=df.drop('Behaviour',axis=1)

In [7]:
y=df['Behaviour']

In [8]:
y

0      1
1      1
2      1
3      1
4      1
      ..
307    1
308    1
309    1
310    1
311    1
Name: Behaviour, Length: 312, dtype: int64

In [9]:
y.value_counts()

1    259
0     53
Name: Behaviour, dtype: int64

In [10]:
X.shape


(312, 5)

In [11]:
y.shape

(312,)

In [12]:
import tensorflow as tf

In [13]:
tf.__version__

'2.9.1'

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import  LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional,GRU,Flatten,Dropout,Dense
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow import keras


In [15]:
!pip install nlpaug

In [16]:
from nlpaug.util import Action
import nlpaug.augmenter.word as word_augmenter

In [17]:
voc_size=5000

In [18]:
messages=X.copy()

In [19]:
messages['Observation'][1]

'Vocal1, Withdrawn'

In [20]:
messages.reset_index(inplace=True)

In [21]:
import nltk
import re
from nltk.corpus import stopwords

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rejee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
# data pre processing
from nltk.stem.porter import PorterStemmer

In [24]:
ps=PorterStemmer()
corpus = []
for i in range(0,len(messages)):
    print(i)
    review=re.sub('[^a-zA-Z]','',messages['Observation'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if not word in stopwords.words ('English')]
    review=''.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [25]:
corpus

['withdrawn',
 'vocalwithdrawn',
 'withdrawn',
 'withdrawn',
 'vocalwithdrawn',
 'withdrawn',
 'communicationvocalwithdrawn',
 'vocal',
 'vocalvoc',
 'repetitionvocalvoc',
 'vocalvoc',
 'vocal',
 'vocal',
 'vocalvoc',
 'repetitionvocalvoc',
 'withdrawn',
 'repetitionvocalvoc',
 'vocal',
 'clothingcommunicationvocalvoc',
 'vocalvoc',
 'vocalvoc',
 'vocalvoc',
 'vocalvoc',
 'vocalvoc',
 'frustrationvoc',
 'vocalvoc',
 'communicationvoc',
 'vocalvoc',
 'communicationrepetitionvoc',
 'communicationvocalvoc',
 'repetit',
 'repetit',
 'commun',
 'repetit',
 'frustrat',
 'withdrawn',
 'frustrationwithdrawn',
 'repetit',
 'frustrationwithdrawn',
 'frustrationwithdrawn',
 'repetit',
 'repetit',
 'withdrawn',
 'repetit',
 'repetit',
 'frustrat',
 'repetit',
 'ambulationfrustr',
 'commun',
 'communicationrepetit',
 'repetit',
 'repetit',
 'frustrationwithdrawn',
 'lost',
 'ambulationfrustrationwithdrawn',
 'frustrat',
 'repetit',
 'repetit',
 'repetit',
 'repetit',
 'frustrationwithdrawn',
 'repe

In [26]:
onehot_repr=[one_hot(words,voc_size,) for words in corpus]
onehot_repr

[[3869],
 [3078],
 [3869],
 [3869],
 [3078],
 [3869],
 [2475],
 [3411],
 [4413],
 [2215],
 [4413],
 [3411],
 [3411],
 [4413],
 [2215],
 [3869],
 [2215],
 [3411],
 [150],
 [4413],
 [4413],
 [4413],
 [4413],
 [4413],
 [1043],
 [4413],
 [4556],
 [4413],
 [399],
 [266],
 [4929],
 [4929],
 [4549],
 [4929],
 [1251],
 [3869],
 [882],
 [4929],
 [882],
 [882],
 [4929],
 [4929],
 [3869],
 [4929],
 [4929],
 [1251],
 [4929],
 [628],
 [4549],
 [4705],
 [4929],
 [4929],
 [882],
 [4942],
 [4088],
 [1251],
 [4929],
 [4929],
 [4929],
 [4929],
 [882],
 [4929],
 [4929],
 [4929],
 [1251],
 [4929],
 [4942],
 [3372],
 [4577],
 [882],
 [4755],
 [2104],
 [628],
 [4929],
 [4929],
 [4755],
 [4929],
 [4929],
 [4929],
 [882],
 [4942],
 [3078],
 [3078],
 [4892],
 [3078],
 [3078],
 [3869],
 [3303],
 [3779],
 [4413],
 [1533],
 [2244],
 [1288],
 [3779],
 [3078],
 [3078],
 [391],
 [2842],
 [882],
 [3303],
 [45],
 [4507],
 [1288],
 [3779],
 [3779],
 [3563],
 [3678],
 [3411],
 [705],
 [45],
 [3411],
 [3078],
 [3411],
 [

In [27]:
sent_length=10
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...    0    0 3869]
 [   0    0    0 ...    0    0 3078]
 [   0    0    0 ...    0    0 3869]
 ...
 [   0    0    0 ...    0    0 4929]
 [   0    0    0 ...    0    0 4929]
 [   0    0    0 ...    0    0 4929]]


In [28]:
len(embedded_docs),y.shape

(312, (312,))

In [29]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [30]:
X_final.shape

(312, 10)

In [31]:
y_final.shape

(312,)

In [32]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.3,random_state=42)

In [33]:
embedding_vector_features=60
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.1))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 60)            300000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              128800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 429,001
Trainable params: 429,001
Non-trainable params: 0
_________________________________________________________________
None


In [34]:
len(embedded_docs),y.shape

(312, (312,))

In [35]:
 
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [36]:
X_final.shape,y_final.shape

((312, 10), (312,))

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [38]:
from tensorflow.keras.utils import plot_model

In [39]:
print(tf.keras.__version__)

2.9.0


In [40]:
model1.fit(X_train,y_train,validation_data = (X_test,y_test),epochs=15,batch_size=100)

Epoch 1/15
3/3 [==============================] - 5s 547ms/step - loss: 0.6826 - accuracy: 0.6890 - val_loss: 0.6423 - val_accuracy: 0.8252
Epoch 2/15
3/3 [==============================] - 0s 96ms/step - loss: 0.6308 - accuracy: 0.8325 - val_loss: 0.5870 - val_accuracy: 0.8252
Epoch 3/15
3/3 [==============================] - 0s 36ms/step - loss: 0.5712 - accuracy: 0.8325 - val_loss: 0.5214 - val_accuracy: 0.8252
Epoch 4/15
3/3 [==============================] - 0s 44ms/step - loss: 0.5046 - accuracy: 0.8325 - val_loss: 0.4667 - val_accuracy: 0.8252
Epoch 5/15
3/3 [==============================] - 0s 103ms/step - loss: 0.4547 - accuracy: 0.8325 - val_loss: 0.4863 - val_accuracy: 0.8252
Epoch 6/15
3/3 [==============================] - 0s 43ms/step - loss: 0.4771 - accuracy: 0.8325 - val_loss: 0.4911 - val_accuracy: 0.8252
Epoch 7/15
3/3 [==============================] - 0s 47ms/step - loss: 0.4672 - accuracy: 0.8325 - val_loss: 0.4609 - val_accuracy: 0.8252
Epoch 8/15
3/3 [=========

In [41]:
pred = (model1.predict(X_test) > 0.5).astype("int32")

4/4 [==============================] - 1s 7ms/step


In [42]:
loss, acc = model1.evaluate(X_test, y_test)

4/4 [==============================] - 0s 6ms/step - loss: 0.4424 - accuracy: 0.8252


In [43]:
acc = np.round(acc, 2) * 100 
loss = np.round(loss, 2) * 100

print(f"Accuracy on unseen data is: { acc } %")
print(f'Loss on unseen data is: { loss } %')

Accuracy on unseen data is: 83.0 %
Loss on unseen data is: 44.0 %


In [44]:
from sklearn.metrics import confusion_matrix

In [45]:
confusion_matrix(y_test,pred)

array([[ 0, 18],
       [ 0, 85]], dtype=int64)

In [46]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred)

0.8252427184466019

In [47]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        18
           1       0.83      1.00      0.90        85

    accuracy                           0.83       103
   macro avg       0.41      0.50      0.45       103
weighted avg       0.68      0.83      0.75       103



C:\Users\rejee\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rejee\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rejee\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
